# Import Packages

In [1]:
# Default packages
import os
from os.path import join
from datetime import date, datetime, timedelta
from joblib import load, dump

# Set current directory to "src"
os.chdir(join(os.getcwd(), os.pardir, "src"))
print(f"Current working directory: {os.getcwd()}")

# Installed packages
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.backend import clear_session
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Defined modules
from util.util import *
# from config.config import *
from process.dataloader import DataTL
from process.embed_gen import EmbedGenTL
from process.evaluate import *
from models import lstm_vae, cnn_vae
from models.lstm_ae import lstm_autoencoder
from models.lstm import basic_lstm
from visualize import plot

# Set TF log level to minimum
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

# Check GPU availability
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Current working directory: e:\Repositories\PCovNet\src
Found GPU at: /device:GPU:0


# Process Config

In [2]:
config = {
    'EXP_NAME': "trained_model_healthy_phase1",
    'EXP_PHASE': "phase1",
    'EXP_GROUP': "healthy",
    'AUGMENT_DATA': True,
    'LEN_WIN': 24,
    'N_WIN': 14,
    'LATENT_DIM': 6,
    'BATCH_SIZE': 64,
    'VAL_SPLIT': 0.05,
    'LEARNING_RATE': 0.0002,
    'EPOCH': 1000,
    'PATIENCE': 10,
}



In [3]:
# Dictionaries of dataset paths
DATA_DIR_DICT = {
    'phase1': join(os.getcwd(), os.pardir, "data", "raw", "phase1"),
    'phase2': join(os.getcwd(), os.pardir, "data", "raw", "phase2")
}
INFO_DIR_DICT = {
    'covid_phase1': join(os.getcwd(), os.pardir, "data", "external", "covid_phase1_info.csv"),
    'covid_phase2': join(os.getcwd(), os.pardir, "data", "external", "covid_phase2_info.csv"),
    'healthy_phase1': join(os.getcwd(), os.pardir, "data", "external", "healthy_phase1_info.csv"),
    'non-covid_phase1': join(os.getcwd(), os.pardir, "data", "external", "non-covid_phase1_info.csv"),
}

# Import subject info
subject_info = pd.read_csv(
    INFO_DIR_DICT[f"{config['EXP_GROUP']}_{config['EXP_PHASE']}"])


# Assign experiment directory
config['EXP_DIR'] = join(os.getcwd(), os.pardir, "experiment",
                         f"{config['EXP_NAME']}_{config['LEN_WIN']}_{config['N_WIN']}")
# timestamp = datetime.now().strftime(f"%Y-%m-%d %H-%M__")
# config['EXP_DIR'] = join(os.getcwd(), os.pardir, "experiment",
#                          timestamp + config['EXP_NAME'])
handle_dir(config['EXP_DIR'])


# Add DATA_DIR
config['DATA_DIR'] = DATA_DIR_DICT[config['EXP_PHASE']]


# Export config
export_json(config, join(config['EXP_DIR'], "config.json"),
            print_json=True)

# Start logging
with open(join(config['EXP_DIR'], "log.txt"), 'w', encoding='utf-8') as f:
    for key, value in config.items():
        f.write(f"{key}: {value}\n")
    f.write("\n\n")


config
{
    "EXP_NAME": "trained_model_healthy_phase1",
    "EXP_PHASE": "phase1",
    "EXP_GROUP": "healthy",
    "AUGMENT_DATA": true,
    "LEN_WIN": 24,
    "N_WIN": 14,
    "LATENT_DIM": 6,
    "BATCH_SIZE": 64,
    "VAL_SPLIT": 0.05,
    "LEARNING_RATE": 0.0002,
    "EPOCH": 1000,
    "PATIENCE": 10,
    "EXP_DIR": "e:\\Repositories\\PCovNet\\src\\..\\experiment\\trained_model_healthy_phase1_24_14",
    "DATA_DIR": "e:\\Repositories\\PCovNet\\src\\..\\data\\raw\\phase1"
}


# Import & Process Data

In [4]:
if 'Symptom Onset' in subject_info.columns:
    subject_info.drop(['Symptom Onset'], axis=1)

data_obj_path = join(config['EXP_DIR'],
                     f"{config['LEN_WIN']}-{config['N_WIN']}_data.joblib")

if not os.path.isfile(data_obj_path):
    # Prepare data
    data = DataTL(config, subject_info)

    # Print data info
    data.print_info()

    # Export data object
    dump(data, data_obj_path)
else:
    data = load(data_obj_path)


100%|██████████| 63/63 [01:43<00:00,  1.65s/it]

            Subject Info
            Phase:                phase1
            Group:                healthy
            
            Dataset Shape
            VAE Train:            (55208, 24, 1)
            VAE Train Aug:        (441664, 24, 1)
            LSTM Train:           (35739, 14, 24, 1)
        


# VAE Model

#### Assign and Compile VAE Model

In [5]:
# Get VAE model
vae_model = cnn_vae.VAE(n_timesteps=config['LEN_WIN'],
                        n_channels=data.train_dataset_vae.shape[-1],
                        latent_dim=config['LATENT_DIM'])
vae_model.compile(loss=tf.losses.MeanSquaredError(),
                  optimizer=tf.optimizers.Adam(
                      learning_rate=config['LEARNING_RATE']),
                  metrics=[tf.metrics.MeanSquaredError()])

# Show VAE model summary
print("\nVAE Model Summary")
print("=================", end="\n\n")
vae_model.print_summary()



VAE Model Summary

Model: "Encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input (InputLayer)     [(None, 24, 1)]      0           []                               
                                                                                                  
 encoder1 (Conv1D)              (None, 12, 128)      512         ['encoder_input[0][0]']          
                                                                                                  
 encoder2 (Conv1D)              (None, 6, 64)        24640       ['encoder1[0][0]']               
                                                                                                  
 encoder3 (Conv1D)              (None, 3, 32)        6176        ['encoder2[0][0]']               
                                                                        

#### Train VAE Model

In [6]:
# Assign checkpoint paths
vae_ckpt_path = join(
    config['EXP_DIR'], "vae_checkpoint", "ckpt")

# Callbacks for VAE
early_stopping_callback = EarlyStopping(monitor='val_loss',
                                        patience=config['PATIENCE'],
                                        mode='min',
                                        restore_best_weights=True)
checkpoint_callback = ModelCheckpoint(vae_ckpt_path,
                                      monitor='val_loss',
                                      verbose=0,
                                      mode='min',
                                      save_best_only=True,
                                      save_weights_only=True)

if not os.path.isfile(vae_ckpt_path + ".index"):
    # Train VAE
    vae_history = vae_model.fit(data.train_dataset_vae,
                                validation_split=config['VAL_SPLIT'],
                                batch_size=config['BATCH_SIZE'],
                                epochs=config['EPOCH'],
                                shuffle=False,
                                verbose=1,
                                callbacks=[early_stopping_callback, checkpoint_callback])

    # Export model history
    export_history(vae_history, join(
        config['EXP_DIR'], "vae_history.csv"))

    # Plot loss curve
    # print("\nVAE Loss Curve")
    # print("==============", end="\n")
    plot.loss_curve(config, vae_history, ref="_VAE", save_plot=True,
                    close_plot=True)

else:
    vae_model.load_weights(vae_ckpt_path)
    print("VAE model weights loaded from:")
    print(vae_ckpt_path)


Epoch 1/1000
820/820 [==============================] - 114s 132ms/step - loss: 22.9381 - reconst_loss: 22.1067 - kl_loss: 0.8314 - val_loss: 19.3333 - val_reconst_loss: 17.5032 - val_kl_loss: 1.8301
Epoch 2/1000
820/820 [==============================] - 54s 66ms/step - loss: 16.6016 - reconst_loss: 15.1213 - kl_loss: 1.4803 - val_loss: 16.3789 - val_reconst_loss: 14.8294 - val_kl_loss: 1.5496
Epoch 3/1000
820/820 [==============================] - 42s 51ms/step - loss: 15.9902 - reconst_loss: 14.6892 - kl_loss: 1.3010 - val_loss: 15.9997 - val_reconst_loss: 14.8071 - val_kl_loss: 1.1926
Epoch 4/1000
820/820 [==============================] - 43s 53ms/step - loss: 15.7754 - reconst_loss: 14.5808 - kl_loss: 1.1946 - val_loss: 15.8346 - val_reconst_loss: 14.5868 - val_kl_loss: 1.2477
Epoch 5/1000
820/820 [==============================] - 46s 57ms/step - loss: 15.6060 - reconst_loss: 14.4672 - kl_loss: 1.1388 - val_loss: 15.6191 - val_reconst_loss: 14.4894 - val_kl_loss: 1.1297
Epoch 6/

# Generate Embeddings

In [7]:
if not os.path.isfile(join(config['EXP_DIR'], f"vae_embeddings.joblib")):
    # Get embedding dataset
    embed_gen = EmbedGenTL(config, vae_model,
                           data, verbose=True)

    # Save embed_gen object
    dump(embed_gen, join(config['EXP_DIR'],
                         f"vae_embeddings.joblib"))
else:
    # Load embed_gen object
    embed_gen = load(join(config['EXP_DIR'], f"vae_embeddings.joblib"))
    print("Embeddings loaded from:")
    print(join(config['EXP_DIR'], f"vae_embeddings.joblib"))


Calculating embeddings for train dataset... 


100%|██████████| 35739/35739 [08:31<00:00, 69.82it/s] 


# LSTM Model for Embeddings

#### Assign and Compile LSTM Model

In [8]:
# Get LSTM MODEL
lstm_model = lstm_autoencoder(n_timesteps=config['N_WIN'] - 1,
                              n_features=config['LATENT_DIM'])
lstm_model.compile(loss=tf.losses.MeanSquaredError(),
                   optimizer=tf.optimizers.Adam(
                       learning_rate=config['LEARNING_RATE']),
                   metrics=['mse'])

# Show LSTM model summary
print("\nLSTM Model Summary")
print("==================", end="\n\n")
lstm_model.summary()



LSTM Model Summary

Model: "LSTM_Autoencoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 13, 6)]           0         
                                                                 
 encoder1 (LSTM)             (None, 13, 128)           69120     
                                                                 
 encoder2 (LSTM)             (None, 64)                49408     
                                                                 
 repeat_vec (RepeatVector)   (None, 13, 64)            0         
                                                                 
 decoder1 (LSTM)             (None, 13, 64)            33024     
                                                                 
 decoder2 (LSTM)             (None, 13, 128)           98816     
                                                                 
 reconst (TimeDistributed)   

#### Train LSTM Model

In [9]:
# Assign checkpoint paths
lstm_ckpt_path = join(
    config['EXP_DIR'], "lstm_checkpoint", "ckpt")
    
# Callbacks for LSTM
early_stopping_callback = EarlyStopping(monitor='val_loss',
                                        patience=config['PATIENCE'],
                                        mode='min',
                                        restore_best_weights=True)
checkpoint_callback = ModelCheckpoint(lstm_ckpt_path,
                                      monitor='val_loss',
                                      verbose=0,
                                      mode='min',
                                      save_best_only=True,
                                      save_weights_only=True)

if not os.path.isfile(lstm_ckpt_path + ".index"):
    # Train LSTM
    lstm_history = lstm_model.fit(embed_gen.x_train, embed_gen.y_train,
                                  validation_split=0.05,
                                  batch_size=config['BATCH_SIZE'],
                                  epochs=config['EPOCH'],
                                  callbacks=[
                                      early_stopping_callback, checkpoint_callback],
                                  verbose=1)

    # Export model history
    export_history(lstm_history, join(
        config['EXP_DIR'], data.id + "_lstm_history.csv"))

    # Plot loss curve
    # print("\nLSTM Loss Curve")
    # print("===============", end="\n")
    plot.loss_curve(config, lstm_history, ref=data.id + "_LSTM", save_plot=True,
                    close_plot=True)
else:
    lstm_model.load_weights(lstm_ckpt_path)
    print("LSTM model weights loaded from:")
    print(lstm_ckpt_path)


Epoch 1/1000
531/531 [==============================] - 35s 49ms/step - loss: 0.8665 - mse: 0.8665 - val_loss: 0.8330 - val_mse: 0.8330
Epoch 2/1000
531/531 [==============================] - 28s 53ms/step - loss: 0.7936 - mse: 0.7936 - val_loss: 0.7725 - val_mse: 0.7725
Epoch 3/1000
531/531 [==============================] - 32s 60ms/step - loss: 0.7389 - mse: 0.7389 - val_loss: 0.7235 - val_mse: 0.7235
Epoch 4/1000
531/531 [==============================] - 30s 56ms/step - loss: 0.6947 - mse: 0.6947 - val_loss: 0.6854 - val_mse: 0.6854
Epoch 5/1000
531/531 [==============================] - 34s 64ms/step - loss: 0.6641 - mse: 0.6641 - val_loss: 0.6587 - val_mse: 0.6587
Epoch 6/1000
531/531 [==============================] - 34s 64ms/step - loss: 0.6382 - mse: 0.6382 - val_loss: 0.6315 - val_mse: 0.6315
Epoch 7/1000
531/531 [==============================] - 32s 61ms/step - loss: 0.6175 - mse: 0.6175 - val_loss: 0.6135 - val_mse: 0.6135
Epoch 8/1000
531/531 [==========================